# Words

More than anything, getting the tokenization of texts done correctly 

In [12]:
# Load the CSV into a dataframe and then place all the texts into a list:

import pandas

with open('../data/Release_v0/TEDonly_final.csv') as f:
    colnames = f.readline().strip().split(",")

print(colnames)

df = pandas.read_csv('../data/Release_v0/TEDonly_final.csv', names=colnames)
print(df.shape)

texts = df.text.tolist()

['rowID', 'Talk_ID', 'public_url', 'speaker_name', 'headline', 'description', 'event', 'duration', 'published', 'tags', 'views', 'text']
(2657, 12)


In [15]:
# Looking for empty texts here:
for text in texts:
    if len(text) < 10:
        print(texts.index(text))

0
